In [14]:
# Block 1: Installation & Setup
# !pip install google-adk google-genai

import os
import textwrap
from google.adk.agents import Agent
from google.adk.tools import AgentTool
from google.adk.runners import InMemoryRunner
from google.genai import types
from google.adk.models.google_llm import Gemini
import nest_asyncio 
nest_asyncio.apply()


# --- CONFIGURATION ---
# Replace with your actual API Key
os.environ["GOOGLE_API_KEY"] = "<GOOGLE_API_KEY>"

print("✅ ADK Libraries Imported Successfully")

✅ ADK Libraries Imported Successfully


In [15]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429,500,503,504]
)

print("retry config created")

retry config created


In [16]:
# Block 2: Define the Sub-Agents (The Specialists)
# We define these as standalone agents. Note the 'description' field
# is CRITICAL here—it tells the Orchestrator when to use this agent.

# 1. Crisis Agent (Safety First)
crisis_agent = Agent(
    name="crisis_specialist",
    model=Gemini(
        model="gemini-2.5-flash", retry_options=retry_config),
    description="Use this tool for ANY input mentioning self-harm, suicide, severe distress, or 'ending it'.",
    instruction=textwrap.dedent("""
        You are a Crisis Intervention Specialist.
        Your ONLY goal is to provide immediate, empathetic safety resources.
        - If a user mentions self-harm, suicide, or severe distress, provide the '988' number.
        - Be warm but firm on safety.
        - Do NOT offer meditation techniques; offer human help.
        - Keep responses short and directive.
    """)
)

# 2. Coach Agent (The Meditation Teacher)
coach_agent = Agent(
    name="meditation_coach",
    model=Gemini(
        model="gemini-2.5-flash", retry_options=retry_config),
    description="Use this tool when the user wants to practice a technique (breathing, body scan) or reduce anxiety.",
    instruction=textwrap.dedent("""
        You are a Mindfulness Coach.
        Your goal is to guide the user through a specific technique.
        - If asked for a 'breathing exercise', provide a 4-7-8 breath guide.
        - If asked for 'grounding', provide the 5-4-3-2-1 technique.
        - Use a calm, slow, and soothing tone.
        - Break instructions into small, readable steps.
    """)
)

# 3. Educator Agent (The Professor)
educator_agent = Agent(
    name="mindfulness_professor",
    model=Gemini(
        model="gemini-2.5-flash", retry_options=retry_config),
    description="Use this tool for theoretical questions (e.g., 'What is mindfulness?', 'How does it affect the brain?').",
    instruction=textwrap.dedent("""
        You are a Mindfulness Educator (Academic).
        - Explain concepts like 'Neuroplasticity', 'Dopamine', or 'Vagus Nerve'.
        - Use scientific terms but explain them simply.
        - Correct misconceptions about mindfulness.
        - Do not guide meditations; only explain the theory.
    """)
)

print("✅ Sub-Agents Initialized")

✅ Sub-Agents Initialized


In [17]:
# Block 3: The Orchestrator (The Hub)
# Instead of writing functions, we wrap the sub-agents in AgentTool.
# The Orchestrator sees these as tools it can "call".

orchestrator = Agent(
    name="mindfulness_orchestrator",
    model=Gemini(
        model="gemini-2.5-pro", retry_options=retry_config), # Stronger reasoning model for routing
    # We wrap the sub-agents as tools here
    tools=[
        AgentTool(agent=crisis_agent),
        AgentTool(agent=coach_agent),
        AgentTool(agent=educator_agent)
    ],
    instruction=textwrap.dedent("""
        You are the central interface for a Mindfulness Application.
        
        Your Responsibility:
        1. Analyze the user's input carefully.
        2. Route the request to the appropriate specialist tool.
        3. Do NOT answer the question yourself. ALWAYS use a tool.
        
        Output Rules:
        - Once the tool provides the answer, present it naturally to the user.
        - If the user says "Hello" or general chit-chat, welcome them and ask how they are feeling to determine the right tool.
    """)
)
runner = InMemoryRunner(agent=orchestrator, app_name="mindfulness_app")
await runner.session_service.create_session(app_name="mindfulness_app",user_id="test_user",session_id="test_session")

print("✅ Runner Initialized and Session 'test_session' Created")

print("✅ Runner Initialized")
print("✅ Orchestrator Ready with Sub-Agent Tools")

✅ Runner Initialized and Session 'test_session' Created
✅ Runner Initialized
✅ Orchestrator Ready with Sub-Agent Tools


In [18]:
# %%
# Block 4: Execution Loop (Final Robust Fix)

import asyncio
from google.genai import types

async def run_chat_async(user_input):
    print(f"\n👤 User: {user_input}")
    print("🤖 Orchestrator is processing...")

    user_msg = types.Content(parts=[types.Part.from_text(text=user_input)])
    # runner = InMemoryRunner(agent=orchestrator)
    try:
        final_reply = None

        async for step in runner.run_async(
            new_message=user_msg,
            user_id="test_user",
            session_id="test_session"
        ):
            if step.content and step.content.parts:
                for part in step.content.parts:
                    if hasattr(part, "text") and part.text:
                        final_reply = part.text  # last text becomes final output

        print("Final:", final_reply)

    except Exception as e:
        print(f"\n❌ Error during execution: {e}")

# ... (Execution calls remain the same) ...
# --- EXECUTION ---
# No need to change these calls, they are now robust.
print("\n--- TEST SCENARIOS (Await Execution) ---\n")

# Scenario 1: Educational Question
await run_chat_async("What actually happens to my brain when I meditate?")

# Scenario 2: Request for Practice
await run_chat_async("I'm feeling super stressed before my exam. Can you help me relax?")

# Scenario 3: Safety/Crisis (Critical Check)
await run_chat_async("I don't see the point of living anymore. I want to end it.")


--- TEST SCENARIOS (Await Execution) ---


👤 User: What actually happens to my brain when I meditate?
🤖 Orchestrator is processing...
Final: That's a fantastic question. Here's what science tells us is happening in your brain when you meditate:

**1. The "Me Center" Quiets Down:** You have a brain network called the Default Mode Network (DMN) that's active when your mind wanders or thinks about yourself. Meditation helps to decrease activity in this area, which is linked to a reduction in worry and self-centered thoughts.

**2. The "Assessment Center" Shrinks:** The amygdala, your brain's fear and anxiety hub, has been shown to decrease in gray matter density with regular meditation. This means your "fight or flight" response becomes less reactive, and you're less likely to be overwhelmed by stress.

**3. The "Executive Center" Gets Stronger:** The prefrontal cortex, which is responsible for focus, concentration, and decision-making, actually thickens. This helps you to better regulat